In [1]:
import boto3
import geojson
import pyarrow as pa
import pandas as pd
import geopandas as gpd

from typing import ClassVar, Type
from datetime import datetime
from pyarrow.fs import S3FileSystem
from dataclasses import dataclass

from pyiceberg.catalog import load_catalog
from pyiceberg.catalog.glue import GlueCatalog
from pyiceberg.table import StaticTable, DataScan
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.io.pyarrow import pyarrow_to_schema
from pyiceberg.transforms import IdentityTransform, DayTransform, YearTransform
from pyiceberg.expressions import EqualTo, GreaterThanOrEqual
from pyiceberg.schema import Schema
from pyiceberg.table import Table
from pyiceberg import types as T

from geojson.utils import coords
from shapely.geometry import shape, Polygon, LinearRing, box, mapping

In [2]:
glue_catalog = load_catalog(
    'default',
    type='glue',
    profile='default',
    region_name='us-east-1',
    client={
        'region': 'us-east-1',
    }
)

In [3]:
# glue_catalog.create_namespace_if_not_exists("db_ice")

In [4]:
glue_catalog.list_namespaces()

[('base',), ('db_ice',), ('default',), ('sagemaker_featurestore',)]

In [5]:
DATABASE = "db_ice"

In [6]:
glue_catalog.list_tables(DATABASE)

[('db_ice', 'tb_controle_imagem'),
 ('db_ice', 'tb_controle_ingestao'),
 ('db_ice', 'tb_imagem'),
 ('db_ice', 'tb_poligono')]

## Create table

In [ ]:
tb_name        = "tb_imagem"
table_location = f"s3://data-us-east-1-891377318910/db_ice/{tb_name}" 

schema = Schema(
    T.NestedField(1, "id_poligono", T.StringType(), required=True),
    T.NestedField(2, "data_imagem", T.StringType(), required=True),
    T.NestedField(3, "no_data", T.FloatType(), required=False),
    T.NestedField(4, "saturated_or_defective", T.FloatType(), required=False),
    T.NestedField(5, "dark_area_pixels", T.FloatType(), required=False),
    T.NestedField(6, "cloud_shadows", T.FloatType(), required=False),
    T.NestedField(7, "vegetation", T.FloatType(), required=False),
    T.NestedField(8, "bare_soils", T.FloatType(), required=False),
    T.NestedField(9, "water", T.FloatType(), required=False),
    T.NestedField(10, "clouds_low_probability_or_unclassified", T.FloatType(), required=False),
    T.NestedField(11, "clouds_medium_probability", T.FloatType(), required=False),
    T.NestedField(12, "clouds_high_probability", T.FloatType(), required=False),
    T.NestedField(13, "thin_cirrus", T.FloatType(), required=False),
    T.NestedField(14, "snow_or_ice", T.FloatType(), required=False),
    identifier_field_ids=[1, 2],
    field_ids=[1, 2]
)

In [9]:
table = glue_catalog.create_table(
    identifier=(DATABASE, tb_name),
    schema=schema,
    location=table_location,
    sort_order=SortOrder(
        fields=[
            SortField(1, IdentityTransform(), ascending=True),
        ]
    ),
    properties={
        "format-version": "2",
        "write.format.default": "parquet",
        "write.parquet.compression-codec": "snappy",
        "write.parquet.page-size": "1048576",  # 1 MB
        "write.parquet.dictionary-enabled": "true",
        "write.parquet.dictionary-page-size": "1048576",  # 1 MB
        "write.parquet.block-size": "268435456",  # 256 MB
        "write.parquet.enable-encoding": "true",
        "write.parquet.enable-statistics": "true",
        "write.parquet.enable-bloom-filter": "true",
        "write.parquet.bloom-filter-encoding": "utf8",
    }
)

## Insert data

In [10]:
table = glue_catalog.load_table((DATABASE, tb_name))

In [11]:
table.schema

<bound method Table.schema of tb_imagem_2(
  1: id_poligono: required string,
  2: data_imagem: required string,
  3: no_data: optional float,
  4: saturated_or_defective: optional float,
  5: dark_area_pixels: optional float,
  6: cloud_shadows: optional float,
  7: vegetation: optional float,
  8: bare_soils: optional float,
  9: water: optional float,
  10: clouds_low_probability_or_unclassified: optional float,
  11: clouds_medium_probability: optional float,
  12: clouds_high_probability: optional float,
  13: thin_cirrus: optional float,
  14: snow_or_ice: optional float
),
partition by: [],
sort order: [1 ASC NULLS FIRST],
snapshot: null>

In [10]:
arrow_schema = pa.schema([
    pa.field("id_poligono", pa.string(), nullable=False),
    pa.field("data_imagem", pa.string(), nullable=False),
    pa.field("no_data", pa.float32(), nullable=True),
    pa.field("saturated_or_defective", pa.float32(), nullable=True),
    pa.field("dark_area_pixels", pa.float32(), nullable=True),
    pa.field("cloud_shadows", pa.float32(), nullable=True),
    pa.field("vegetation", pa.float32(), nullable=True),
    pa.field("bare_soils", pa.float32(), nullable=True),
    pa.field("water", pa.float32(), nullable=True),
    pa.field("clouds_low_probability_or_unclassified", pa.float32(), nullable=True),
    pa.field("clouds_medium_probability", pa.float32(), nullable=True),
    pa.field("clouds_high_probability", pa.float32(), nullable=True),
    pa.field("thin_cirrus", pa.float32(), nullable=True),
    pa.field("snow_or_ice", pa.float32(), nullable=True),
])

In [ ]:
table_values = [
    {
        "id_poligono": "id_101", 
        "data_imagem": "2025-08-29",

    },
]

In [ ]:
pa_table = pa.Table.from_pylist(
    table_values,
    schema=arrow_schema
)

In [17]:
pa_table.to_pandas().head()

,id_poligono,poligono,area_poligono,caminho_s3
0,id_101,"POLYGON((-47.9296875 -15.601025390625,-47.8125...",100,s3://data-us-east-1-891377318910/geospatial/mv...


In [18]:
table.append(pa_table)